In [ ]:
%%tsql -artifact Fabcon_database -type SQLDatabase


 SELECT P.ProductID, P.Name as ProductName, K.Description Into SalesLT.dim_Product
FROM SalesLT.Product AS P
INNER JOIN (
SELECT PPD.ProductModelID, PD.Description

FROM SalesLT.ProductModelProductDescription AS PPD
INNER JOIN SalesLT.ProductDescription AS PD
ON PD.ProductDescriptionID = PPD.ProductDescriptionID
 AND LEN(Description)<=70
) AS K
ON P.ProductModelID = K.ProductModelID





In [ ]:
%%tsql -artifact Fabcon_database -type SQLDatabase
-- Create a master key for encryption with the specified password

CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Smoothie@123';

-- Create a database scoped credential for accessing a specific HTTPS endpoint
CREATE DATABASE SCOPED CREDENTIAL [https://AI_ENDPOINT_SERVERNAME.openai.azure.com/] 
WITH IDENTITY = 'HTTPEndpointHeaders',
 SECRET = '{"api-key": "YOUR_OPENAI_KEY"}';

-- Retrieve information about the database scoped credential with the specified name
SELECT * 
FROM sys.database_scoped_credentials
WHERE name = 'https://AI_ENDPOINT_SERVERNAME.openai.azure.com';

In [ ]:
%%tsql -artifact Fabcon_database -type SQLDatabase
create or alter procedure [SalesLT].[get_embedding]
    @inputText nvarchar(max),
    @embedding nvarchar(max) output
as
begin try
    declare @retval int;
    declare @payload nvarchar(max) = json_object('input': @inputText);
    declare @response nvarchar(max);

    -- Build headers dynamically
    DECLARE @headers NVARCHAR(MAX) = JSON_OBJECT(
        'Content-Type': 'application/json',
        'api-key':N'YOUR_OPENAI_KEY'
    );

    exec @retval = sp_invoke_external_rest_endpoint
        @url =  N'https://AI_ENDPOINT_SERVERNAME.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
        @method = 'POST',
        @headers = @headers,
        @payload = @payload,
        @response = @response output;
end try
begin catch
    select 
        'SQL' as error_source,
        error_number() as error_code,
        error_message() as error_message;
    return;
end catch;

if (@retval != 0)
begin
    select 
        'OPENAI' as error_source,
        json_value(@response, '$.result.error.code') as error_code,
        json_value(@response, '$.result.error.message') as error_message,
        @response as error_response;
    return;
end;

-- Extract and store the embedding as JSON text
declare @re nvarchar(max) = json_query(@response, '$.result.data[0].embedding');
set @embedding = @re;



In [8]:
%%tsql -artifact Fabcon_database -type SQLDatabase
-- Create a table named Product_Embeddings to hold product names, descriptions, and embeddings
--truncate table SalesDB.Product_Embeddings
CREATE TABLE SalesLT.Product_Embeddings (
    ProductName NVARCHAR(MAX),
	Description NVARCHAR (MAX),
    EmbeddedName VECTOR(1536)
);

-- Create a table named ChatMessages to store chat message details
CREATE TABLE SalesLT.ChatMessages (
    MessageID INT IDENTITY(1,1) PRIMARY KEY,
    UserSessionID UNIQUEIDENTIFIER,
    MessageType NVARCHAR(10),  -- 'User' or 'AI'
    MessageText NVARCHAR(MAX),
    CreatedAt DATETIME DEFAULT GETDATE()
);


-- Declare variables to store product name, description, and embedding
DECLARE @ProductName NVARCHAR(MAX);
DECLARE @Description NVARCHAR(MAX);
DECLARE @Embedding VECTOR(1536);

-- Declare a cursor to iterate over product names and descriptions
DECLARE ProductCursor CURSOR FOR
SELECT ProductName, Description
FROM SalesLT.dim_product;

OPEN ProductCursor;

FETCH NEXT FROM ProductCursor INTO @ProductName, @Description;

WHILE @@FETCH_STATUS = 0
BEGIN
    -- Call the get_embedding procedure to get the embedding for the product name
    EXEC [SalesLT].[get_embedding]
        @inputText = @ProductName,
        @embedding = @Embedding OUTPUT;

    -- Store the product name, description, and embedding in the Product_Embeddings table
    INSERT INTO SalesLT.Product_Embeddings (ProductName, Description, EmbeddedName)
    VALUES (@ProductName, @Description, @Embedding);

    -- Fetch the next row
    FETCH NEXT FROM ProductCursor INTO @ProductName, @Description;
END;

CLOSE ProductCursor;
DEALLOCATE ProductCursor;


In [12]:

%%tsql -artifact Fabcon_database -type SQLDatabase
SELECT  top 100 * FROM SalesLT.Product_Embeddings

In [ ]:

%%tsql -artifact Fabcon_database -type SQLDatabase
CREATE OR ALTER PROCEDURE [SalesLT].[find_relevant_products]
@text NVARCHAR(MAX),
@top INT = 10,
@min_similarity DECIMAL(19,16) = 0.50,
@JsonResults NVARCHAR(MAX) OUTPUT
AS
BEGIN
    -- Check if the search query is NULL, if so, exit the procedure
    IF (@text IS NULL) RETURN;

    DECLARE @RefinedQuery NVARCHAR(MAX);
    DECLARE @LLMResponse NVARCHAR(MAX);
    DECLARE @LLMRetval INT;
    DECLARE @LLMPayload NVARCHAR(MAX);

    -- Prepare the payload for the Language Model API call
    SET @LLMPayload = 
    N'{
        "model": "gpt-4.1",
        "messages": [
            {
                "role": "system",
                "content": "You are an assistant that extracts the core product keyword(s) from a user''s query."
            },
            {
                "role": "user",
                "content": "Extract the key product name or keywords from the following query: ' 
                + REPLACE(@text, '"', '\"') + '"
            }
        ],
        "temperature": 0.5,
        "max_tokens": 20
    }';

    -- Call the external REST endpoint to interact with the Language Model
    EXEC @LLMRetval = sp_invoke_external_rest_endpoint
         @url = 'https://AI_ENDPOINT_SERVERNAME.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview',
         @method = 'POST',
         @credential = [https://AI_ENDPOINT_SERVERNAME.openai.azure.com/],
         @payload = @LLMPayload,
         @response = @LLMResponse OUTPUT;

    -- Extract the refined query from the LLM response JSON
    SET @RefinedQuery = JSON_VALUE(@LLMResponse, '$.result.choices[0].message.content');

    -- If the refined query is NULL or empty, use the original search query
    IF (@RefinedQuery IS NULL OR LEN(@RefinedQuery) = 0)
        SET @RefinedQuery = @text;


    DECLARE @retval INT, @qv VECTOR(1536);

    -- Call the get_embedding stored procedure to get the embedding for the refined query
    EXEC @retval = [SalesLT].get_embedding @RefinedQuery, @qv OUTPUT;

    -- If an error occurred in get_embedding, exit the procedure
    IF (@retval != 0) RETURN;

    -- Retrieve the top similar product embeddings based on the query embedding
    WITH cteSimilarEmbeddings AS (
        SELECT TOP(@top)
            pe.ProductName AS ProductName,
            pe.Description AS Description,
            vector_distance('euclidean', pe.[EmbeddedName], @qv) AS distance
        FROM SalesLT.Product_Embeddings pe
        ORDER BY distance
    )

    -- Select the relevant product information in JSON format based on similarity threshold
    SELECT @JsonResults = (
        SELECT 
            p.ProductName AS ProductName
        FROM cteSimilarEmbeddings se
        INNER JOIN SalesLT.dim_product p ON se.ProductName = p.ProductName
        WHERE 1 - distance >= @min_similarity
        ORDER BY distance
        FOR JSON AUTO, ROOT('search_results')
    );
END;

In [ ]:

%%tsql -artifact Fabcon_database -type SQLDatabase
CREATE OR ALTER PROCEDURE SalesLT.get_ai_response
@UserSessionID UNIQUEIDENTIFIER,
@UserQuery NVARCHAR(MAX),
@AIResponse NVARCHAR(MAX) OUTPUT
AS
BEGIN
    DECLARE @SearchResults NVARCHAR(MAX);

    -- Step 1: Fetch relevant products
    EXEC SalesLT.find_relevant_products @UserQuery, 10, 0.50, @SearchResults OUTPUT;

    -- Step 2: Ensure @SearchResults is not NULL or empty
    IF @SearchResults IS NULL OR LEN(@SearchResults) = 0
        SET @SearchResults = '{"search_results":[]}';  -- Prevent NULL errors

    DECLARE @CleanSearchResults NVARCHAR(MAX);
    SET @CleanSearchResults = REPLACE(@SearchResults, '"', '\"');

    -- Debugging: Display the search results before making API call
    -- SELECT @SearchResults AS DebugSearchResults;

    -- Step 3: Construct JSON payload correctly
    DECLARE @Payload NVARCHAR(MAX);
    SET @Payload = 
    N'{
        "model": "gpt-4.1",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant providing structured product summaries in a clean, readable format. Use bullet points, avoid excessive markdown (like ### or ***), and remove redundant blank lines. Make responses interactive by encouraging the user to choose."
            },
            {
                "role": "user",
                "content": "Based on the following product search results, generate a structured yet conversational summary with emojis, bullet points, and minimal empty lines:\n\n' + @CleanSearchResults + '"
            }
        ],
        "temperature": 0.7,
        "max_tokens": 300
    }';

    -- Debugging: Display the payload before calling Azure OpenAI API
    -- SELECT @Payload AS DebugPayload;

    -- Step 4: Call Azure OpenAI API
    DECLARE @retval INT, @response NVARCHAR(MAX);
    
    EXEC @retval = sp_invoke_external_rest_endpoint
        @url = 'https://AI_ENDPOINT_SERVERNAME.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview',
        @method = 'POST',
        @credential = [https://AI_ENDPOINT_SERVERNAME.openai.azure.com/],
        @payload = @Payload,
        @response = @AIResponse OUTPUT;

    DECLARE @NaturalLanguageResponse NVARCHAR(MAX);
    SET @NaturalLanguageResponse = JSON_VALUE(@AIResponse, '$.result.choices[0].message.content');

    -- Step 5: Split AI response into separate messages and store in ChatMessages
    DECLARE @Pos INT = 1, @Line NVARCHAR(MAX), @Delimiter NVARCHAR(2) = CHAR(10);
    
    -- Step 6: Store the structured response in the database
    INSERT INTO SalesLT.ChatMessages (UserSessionID, MessageType, MessageText)
    VALUES (@UserSessionID, 'User', @UserQuery);

    -- Loop through AI response, splitting it into rows
    WHILE CHARINDEX(@Delimiter, @NaturalLanguageResponse, @Pos) > 0
    BEGIN
        SET @Line = LEFT(@NaturalLanguageResponse, CHARINDEX(@Delimiter, @NaturalLanguageResponse, @Pos) - 1);
        SET @NaturalLanguageResponse = STUFF(@NaturalLanguageResponse, 1, CHARINDEX(@Delimiter, @NaturalLanguageResponse, @Pos), '');

        -- Store each line separately
        INSERT INTO SalesLT.ChatMessages (UserSessionID, MessageType, MessageText)
        VALUES (@UserSessionID, 'AI', @Line);
    END;

    -- Store any remaining text
    IF LEN(@NaturalLanguageResponse) > 0
    BEGIN
        INSERT INTO SalesLT.ChatMessages (UserSessionID, MessageType, MessageText)
        VALUES (@UserSessionID, 'AI', @NaturalLanguageResponse);
    END;

    -- Step 6: Return AI response as multiple rows
    SELECT 
        CASE 
            WHEN MessageType = 'User' THEN 'User: ' + MessageText
            WHEN MessageType = 'AI' THEN 'AI: ' + MessageText
        END AS ChatMessage
    FROM SalesLT.ChatMessages
    WHERE UserSessionID = @UserSessionID
    ORDER BY CreatedAt ASC;
END;



In [1]:
%%tsql -artifact Fabcon_database -type SQLDatabase
--- Question 1: find me phones
DECLARE @UserSessionID UNIQUEIDENTIFIER = NEWID();
DECLARE @AIResponse NVARCHAR(MAX);

--EXEC SalesLT.get_ai_response @UserSessionID, 'Find me all Cranksets', @AIResponse OUTPUT;
--EXEC SalesLT.get_ai_response @UserSessionID, 'Find me all handle bars', @AIResponse OUTPUT;
--EXEC SalesLT.get_ai_response @UserSessionID, 'Find me front wheel with aerodynamic ring', @AIResponse OUTPUT;
EXEC SalesLT.get_ai_response @UserSessionID, 'Aluminium Alloy handle bars', @AIResponse OUTPUT;
--EXEC SalesLT.get_ai_response @UserSessionID, 'Batteries which uses 3 AAA batteries', @AIResponse OUTPUT;
SELECT @AIResponse AS GPT_Response;

